#### Mini Project - Object Detection

**Using SIFT (can't be used in production app for free since algo is patented)**

In [2]:
import cv2
import numpy as np


def sift_detector(new_image, image_template):
    """
    compares the new_image with the image_template
    and returns the number of SIFT matches found between them.
    """
    image_1 = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)
    image_2 = image_template
    
    # SIFT detector object
    sift = cv2.SIFT_create()
    
    # obtains the keypoints and vector descriptors that store info about the keypoints using SIFT
    keypoints_1, descriptors_1 = sift.detectAndCompute(image_1, None)
    keypoints_2, descriptors_2 = sift.detectAndCompute(image_2, None)
    
    # Defines paramters for our Flann matcher
    FLANN_INDEX_KDTREE = 0
    # note that the higher the int assigned to the tree var the more slower and complicated the matching process is.
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 3)
    # specifies the max number of matches that the flann matcher should look for.
    search_params = dict(checks=100)
    
    # code that creates the FLANN matcher
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    
    # code that obtain matches using the K-Nearest Neighbour method.
    # note: matches are similar keypoints found btw images.
    # the var; "matches" = holds the value of the number of similar matches found within both images.
    matches = flann.knnMatch(descriptors_1, descriptors_2, k=2)
    
    # The Lowe Ration test helps in get the right accuracy of matches
    # The lowe Ratio test, stores the good Matches.
    good_matches = []
    for m, n in matches:
        if m.distance < 0.7 * n.distance:
            good_matches.append(m)
    # returns the number of good matches found and returns it.        
    return len(good_matches)

In [1]:
# code that initializes our PC webcam for use
cap = cv2.VideoCapture(0)

# loads the image template, this is the reference image.
# i.e the image viewed by our webcam must be similar to it.
image_template = cv2.imread(r'C:\Users\princ\Documents\Mini openCV projects\image_for_computer_vision\Box_in_scene.png',0)

while True:
    ret, frame = cap.read()
    
    # the height and width of the webcam frame
    height, width = frame.shape[:2]
    
    # defines the Box that the object must be placed in, for matching to be done i.e ROI.
    top_left_x = int(width/3)
    top_left_y = int((height/2)+(height/4))
    bottom_right_x = int((width/3) * 2)
    bottom_right_y = int((height/2) - (height/4))
    
    # draws rectangular windows for the box where the matching process is done.
    cv2.rectangle(frame, (top_left_x, top_left_y), (bottom_right_x, bottom_right_y),255,3)
    
    # code that crops the window of obeservation that we defined above
    cropped = frame[bottom_right_y:top_left_y, top_left_x:bottom_right_x]
    
    # flips the frame orientation horizontally
    # This gives the webcam feed a mirror like effect.
    frame = cv2.flip(frame,1)
    
    # Gets number of SIFT matches
    matches = sift_detector(cropped, image_template)
    
    # that displays status showing the current number of matches.
    cv2.putText(frame, str(matches), (450,450), cv2.FONT_HERSHEY_COMPLEX, 2,(255,0,0), 1)
    
    # that specifies the threshold that indicates the Object detection
    threshold = 20
    
    # if mathches exceed our threshold, then then the object has been detected.
    if matches > threshold:
        cv2.rectangle(frame, (top_left_x, top_left_y),(bottom_right_x, bottom_right_y),(0,0,255),3)
        cv2.putText(frame, 'The Object has been found', (50,50), cv2.FONT_HERSHEY_COMPLEX, 1,(0, 0, 255),3)
        
    cv2.imshow('Object Detector using SIFT', frame)
    if cv2.waitKey(1) == 13:       # specifies that the Enter key is used to stop the program.
        break

        
cap.release()
cv2.destroyAllWindows()
        

**Object Detection Using ORB**

* Since SIFT feature detection algorightm is patented and can't be used for commercial use, we can use the ORB feature detection algorithm.

In [4]:
import cv2
import numpy as np

def ORB_detector(new_image, image_template):
    """
    function that compares the new_image to the template image
    and also returns the number of matches found between them
    """
    
    image_1 = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)
    image_2 = image_template
    
    # creates the ORB detector with 1000 keypoints and a scaling factor of "1.2"
    orb = cv2.ORB_create(1000, 1.2)
    
    # detects keypoints and crealtes vector decriptors for image_1 and image_2.
    keypoints_1, descriptors_1 = orb.detectAndCompute(image_1, None)
    keypoints_2, descriptors_2 = orb.detectAndCompute(image_2, None)

    # creates a matcher using the BRUTE FORCE MATCHER ALGORITHM
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    
    # does the matching using the descriptors of both images.
    # This used descriptors store info about all the keypoints of image_1 and image_2 respectively
    matches = bf.match(descriptors_1, descriptors_2)
    
    # code that sorts the matches based on distance, least distance is better.
    # Note this is not a must but is just good practice.
    matches = sorted(matches, key=lambda val: val.distance)
    
    #code that returns the number of matches found
    return len(matches)

In [6]:
cap = cv2.VideoCapture(0)

# code that loads our template_image/ reference image
image_template = cv2.imread(r'C:\Users\princ\Documents\Mini openCV projects\image_for_computer_vision\Box_in_scene.png',0)

while 1:
    # code that gets webcam images
    ret, frame = cap.read()
    
    # code that gets a height and width of the webcam frame
    height, width = frame.shape[:2]
    
    # defines the Box that contains the region of Interest(ROI).
    top_left_x = int(width /3)
    top_left_y = int((height/2) + (height/4))
    bottom_right_x = int((width/3) * 2)
    bottom_right_y = int((height/2) - (height/4))
    
    # draws the rectangular window of ROI.
    cv2.rectangle(frame, (top_left_x, top_left_y), (bottom_right_x, bottom_right_y),255,4)
    
    # crops the window of obeservation we define above
    cropped_image = frame[bottom_right_y:top_left_y, top_left_x:bottom_right_x]
    
    # flips the image horizontally inorder to get a mirror like effect on our webcam feed
    frame = cv2.flip(frame,1)
    
    # gets the number of ORB matches btw the reference image and the image gotten in the ROI from the webcam
    matches = ORB_detector(cropped_image, image_template)
    
    #  displays shows the current number of matches
    info = "The Matches found = " + str(matches)
    cv2.putText(frame, info, (50,450),cv2.FONT_HERSHEY_COMPLEX,1, (100, 255, 150),2)
    
    # indicates the threshold for object to be confirmed detected
    # note: To get top 1000 matches, 350 is a good min match
    threshold = 250
    
    # if matches exceed our threshold then the object has been detected
    if matches > threshold:
        cv2.rectangle(frame, (top_left_x, top_left_y),(bottom_right_x, bottom_right_y), (255, 0, 100), 2)
        cv2.putText(frame, 'Object Found', (50,50), cv2.FONT_HERSHEY_COMPLEX,2, (100, 200, 40), 3)
        
    cv2.imshow('Object Detection using The ORB detector', frame)
    
    # quits when Enter key is pressed
    if cv2.waitKey(1) == 13:
        break

cap.release()
cv2.destroyAllWindows()